In [133]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
import ast
import pickle

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credit = pd.read_csv('tmdb_5000_credits.csv')

In [4]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [10]:
movies = movies[['title','genres','keywords','overview']]

In [11]:
movies

,title,genres,keywords,overview
0,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."
...,...,...,...,...
4798,El Mariachi,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...
4799,Newlyweds,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],A newlywed couple's honeymoon is upended by th...
4800,"Signed, Sealed, Delivered","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,Shanghai Calling,[],[],When ambitious New York attorney Sam is sent t...


In [7]:
credit.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [13]:
df = movies.merge(credit,on='title')

In [15]:
df.shape

(4809, 7)

In [16]:
df.isnull().sum()

title       0
genres      0
keywords    0
overview    3
movie_id    0
cast        0
crew        0
dtype: int64

In [19]:
df.dropna(inplace=True)

In [79]:
df.head(4)

,title,genres,keywords,overview,movie_id,cast,crew,tags
0,Avatar,"[action, adventur, fantasi, sciencefict]","[cultureclash, futur, spacewar, spacecoloni, s...","[in, the, 22nd, century,, a, parapleg, marin, ...",19995,"[samworthington, zoesaldana, sigourneyweav]",[jamescameron],"[action, adventur, fantasi, sciencefict, cultu..."
1,Pirates of the Caribbean: At World's End,"[adventur, fantasi, action]","[ocean, drugabus, exoticisland, eastindiatradi...","[captain, barbossa,, long, believ, to, be, dea...",285,"[johnnydepp, orlandobloom, keiraknightley]",[goreverbinski],"[adventur, fantasi, action, ocean, drugabus, e..."
2,Spectre,"[action, adventur, crime]","[spi, basedonnovel, secretag, sequel, mi6, bri...","[a, cryptic, messag, from, bond’, past, send, ...",206647,"[danielcraig, christophwaltz, léaseydoux]",[sammend],"[action, adventur, crime, spi, basedonnovel, s..."
3,The Dark Knight Rises,"[action, crime, drama, thriller]","[dccomic, crimefight, terrorist, secretident, ...","[follow, the, death, of, district, attorney, h...",49026,"[christianbal, michaelcain, garyoldman]",[christophernolan],"[action, crime, drama, thriller, dccomic, crim..."


In [26]:
def convert(text):
    text = ast.literal_eval(text)
    l = []
    for i in text:
        l.append(i['name'])
    return l

In [60]:
def cast_convert(text):
    text = ast.literal_eval(text)
    l = []
    count = 0
    for i in text:
        if count == 3:
            break
        l.append(i['name'])
        count += 1
        
    return l

In [61]:
def crew_convert(text):
    text = ast.literal_eval(text)
    l = []
    for i in text:
        if i['department'] == 'Directing' and i['job'] == 'Director':
            l.append(i['name'])
    return l

In [62]:
def cn(l):
    l1 = []
    count = 0
    for i in l:
        if count == 3:
            break
        l1.append(i)
        count += 1
    return l1

In [63]:
def stemmer(lis):
    ps = PorterStemmer()
    l = []
    for i in lis:
        l.append(ps.stem(i))
    return l

In [64]:
def joiner(lis):
    l = []
    for i in lis:
        l.append(i.replace(' ',''))
    return l

In [77]:
def smaller(lis):
    l = []
    for i in lis:
        l.append(i.lower())
    return l

In [28]:
df['genres'] = df['genres'].apply(convert)

In [34]:
df['keywords'] = df['keywords'].apply(convert)

In [50]:
df['cast'] = df['cast'].apply(cn)

In [52]:
df['crew'] = df['crew'].apply(crew_convert)

In [54]:
df['overview'] = df['overview'].apply(lambda x : list(x.split(' ')))

In [57]:
df['genres'] = df['genres'].apply(stemmer)
df['keywords'] = df['keywords'].apply(stemmer)
df['cast'] = df['cast'].apply(stemmer)
df['crew'] = df['crew'].apply(stemmer)
df['overview'] = df['overview'].apply(stemmer)

In [78]:
df['genres'] = df['genres'].apply(smaller)
df['keywords'] = df['keywords'].apply(smaller)
df['cast'] = df['cast'].apply(smaller)
df['crew'] = df['crew'].apply(smaller)
df['overview'] = df['overview'].apply(smaller)

In [ ]:
df['genres'] = df['genres'].apply(joiner)
df['keywords'] = df['keywords'].apply(joiner)
df['cast'] = df['cast'].apply(joiner)
df['crew'] = df['crew'].apply(joiner)
df['overview'] = df['overview'].apply(joiner)

In [80]:
df['tags'] = df['genres'] + df['keywords'] + df['cast'] + df['crew'] + df['overview']

In [81]:
new_df = df[['movie_id','title','tags']]

In [82]:
new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))

C:\Users\gyash\AppData\Local\Temp\ipykernel_19952\3530946481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x : " ".join(x))


In [83]:
cv = CountVectorizer(stop_words=['english'],max_features=5000)

In [87]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [89]:
vector.shape

(4806, 5000)

In [95]:
similarity = cosine_similarity(vector)


In [96]:
similarity.shape

(4806, 4806)

In [129]:
def recommend(name):
    index = new_df[new_df['title'] == name].index[0]
    x = sorted(enumerate(similarity[index]),reverse=True,key=lambda x : x[1])
    l = []
    for i in x[1:6]:
        l.append(i[0])
    return l
    


In [132]:
a = recommend('Avatar')
new_df.iloc[a]

,movie_id,title,tags
1216,440,Aliens vs Predator: Requiem,fantasi action sciencefict thriller horror pre...
507,602,Independence Day,action adventur sciencefict spacecraft patriot...
1089,20856,Aliens in the Attic,adventur comedi famili fantasi sciencefict ali...
300,563,Starship Troopers,adventur action thriller sciencefict moon aste...
2409,679,Aliens,horror action thriller sciencefict android ext...


In [99]:
new_df

,movie_id,title,tags
0,19995,Avatar,action adventur fantasi sciencefict culturecla...
1,285,Pirates of the Caribbean: At World's End,adventur fantasi action ocean drugabus exotici...
2,206647,Spectre,action adventur crime spi basedonnovel secreta...
3,49026,The Dark Knight Rises,action crime drama thriller dccomic crimefight...
4,49529,John Carter,action adventur sciencefict basedonnovel mar m...
...,...,...,...
4804,9367,El Mariachi,action crime thriller unitedstates–mexicobarri...
4805,72766,Newlyweds,comedi romanc edwardburn kerrybishé marshadiet...
4806,231617,"Signed, Sealed, Delivered",comedi drama romanc tvmovi date loveatfirstsig...
4807,126186,Shanghai Calling,danielhenney elizacoup billpaxton danielhsia w...


In [137]:
pickle.dump(new_df,open("movies_list.pkl",'wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))